<a href="https://colab.research.google.com/github/vaidasal/ROCT/blob/master/FilterungTOP_BOTTOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import copy
from sklearn.cluster import KMeans
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit

In [2]:
def readJson(path, type):
    print("Reading Json... ")
    with open(path) as json_file:
        data = json.load(json_file)
        table = pd.read_json(data[type], typ="frame", orient="values")
        df = pd.DataFrame(table)
        df.columns = data[f"C__{type}"]
        return df

Find Start end index

In [3]:
def findStartEnd(df_original):
  df_removed = df_original.drop("Line Start [us]", axis=1).drop("Line End [us]", axis=1).reset_index(drop=True).replace("", float("nan")).astype("float64")

  x = []
  s = []
  xs = []
  for a in range(len(df_removed)):
    ddd = df_removed.iloc[a,:].dropna()
    s.append(np.std(ddd))
    x.append(a)
    xs.append([a, np.std(ddd)])

  max_std = max(s)
  start = 0
  end = df_removed.shape[0]
  for b in range(len(s)):
    if s[b] > max_std/20:
      start = int(b)
      break

  for c in reversed(range(len(s))):
    if s[c] > max_std/20:
      end = int(c)
      break
  
  return start, end

Continuous X Value and XY extration

In [4]:
def create1DXCorrected(df, rob_speed, start, end):
  v_rob = rob_speed
  # Dataframe aufbereiten
  start_time = df_original.iloc[:,0]
  end_time = df_original.iloc[:,1]
  df = df_original.drop([df_original.columns[0], df_original.columns[1]], axis = 1, inplace = False)
  df = df.replace(r'^\s*$', np.nan, regex=True)
  df = df.apply(pd.to_numeric, errors='ignore')
  cols = pd.to_numeric(df.columns)
  df.columns = np.arange(len(df.columns))

  # Zeitstempel und x-Positionen generieren
  arr_time = np.empty([df.shape[0], cols.size])
  arr_oct_pos = np.empty([df.shape[0], cols.size])
  true_start_time = np.empty(df.shape[0])
  true_end_time = np.empty(df.shape[0])
  for i in range(df.shape[0]):
      first = df.iloc[i].first_valid_index()
      last = df.iloc[i].last_valid_index()
      if first != last:
          timestep = (end_time[i] - start_time[i]) / (last - first)
      true_start_time[i] = start_time[i] - (timestep * first)
      true_end_time[i] = end_time[i] + timestep * (cols.size - 1 - last)
  true_start_time = pd.Series(true_start_time).interpolate()
  true_end_time = pd.Series(true_end_time).interpolate()
  for i in range(df.shape[0]):
      arr_time[i] = np.linspace(true_start_time[i], true_end_time[i], cols.size, endpoint=True)
      arr_oct_pos[i] = cols
  arr_xpos = arr_oct_pos + arr_time * v_rob / 1000

  # Zeitstempel nach x-Position sortieren # Beachte: f_sortiert = f_unsortiert(f_reihenfolge)  <<und>>  f_unsortiert = f_sortiert(f_reihenfolge)
  arr_xpos_1d = arr_xpos.flatten()
  yscatter = df.values.flatten()
  reihenfolge = np.argsort(arr_xpos_1d)
  yscatter_x = np.empty(reihenfolge.size)
  arr_xpos_x = np.empty(reihenfolge.size)
  for i in range(reihenfolge.size):
      yscatter_x[i] = yscatter[reihenfolge[i]]
      arr_xpos_x[i] = arr_xpos_1d[reihenfolge[i]]
    
  return (arr_xpos_x, yscatter_x, arr_xpos[start,0], arr_xpos[end,-1])

Creating Grid

In [5]:
def createGrid(x, y, df_original):
  x_min = np.min(x)
  x_max = np.max(x)
  df_original_mod = df_original.drop("Line Start [us]", axis=1).drop("Line End [us]", axis=1).reset_index(drop=True).replace("", float("nan")).astype("float64")
  y_min = df_original_mod.min().min()
  y_max = df_original_mod.max().max()
  rows = len(df_original_mod)

  nx, ny = (300, 200)
  x_space = np.linspace(x_min, x_max, nx)
  y_space = np.flip(np.linspace(y_min, y_max, ny))
  df0 = pd.DataFrame(np.zeros((ny,nx)), columns=x_space, index=y_space)

  return df0, x_space, y_space

Fill Grid

In [6]:
def fillEmptyGrid(grid, x, y, x_space, y_space):
  xy = pd.Series(y, index=x).dropna()
  for index, value in xy.items():
    x_sp = (np.abs(x_space-index)).argmin()
    y_sp = (np.abs(y_space-value)).argmin()
    grid.iloc[y_sp, x_sp] = grid.iloc[y_sp, x_sp] + 1

  return grid

Convert to 3D Array

In [7]:
def gridTo3D(data):
  x = []
  y = []
  z = []


  l = len(data.index)
  x_dummy = min(data.columns)
  x = [x_dummy] * l
  y = list(data.index)
  z = [0] * l


  lc = len(data.columns)
  y_dummy = min(data.index)
  y.extend([y_dummy] * lc)
  x.extend(list(data.columns))
  z.extend([0] * lc)

  for row in data.index:
    for col in data.columns:
      d = data.loc[row,col]
      if d != 0:
        x.append(col)
        y.append(row)
        z.append(d)
  return x,y,z

Noise Filter square

In [8]:
def noiseFilter(df_f, row, col, row_len, col_len, size):
  val = df_f.iloc[row,col]

  fil_size = int(size + val)
  r0 = row - fil_size
  r1 = row + fil_size + 1
  if r0 < 0 : r0 = 0
  if r1 > row_len: r1 = row_len
  c0 = col - fil_size
  c1 = col + fil_size + 1
  if c0 < 0 : c0 = 0
  if c1 > col_len: c1 = col_len

  fil_frame = df_f.iloc[r0:r1, c0:c1]

  frame_sum = fil_frame.sum().sum()
  if val == frame_sum: val = 0

  return val

def filterNoise(df0, size):
  df_fil = copy.deepcopy(df0)
  x_len = len(x)
  df_max = df_fil.max().max()
  col_len = len(df_fil.columns)
  row_len = len(df_fil.index)
  col_range = range(col_len)
  row_range = range(row_len)
  for row in row_range:
    for col in col_range:
      if df_fil.iloc[row,col] == 0:
        pass
      else:
        df_fil.iloc[row,col] = noiseFilter(df_fil, row, col, row_len, col_len, size)

  for index, row in df_fil.iloc[::-1].iterrows():
    if row.sum() == 0:
      df_fil.drop(index=index, inplace=True)
    else:
      break
  return df_fil

KMeans Clustering finding Centers with sum

In [9]:
def clusterAndFindCenter(data):

  # Calculating Sum
  item_list = []
  for index, value in data.sum().items():
    item_list.append([index, value])
  

  Kmean = KMeans(n_clusters=3)
  Kmean.fit(item_list)
  center = pd.DataFrame(Kmean.cluster_centers_).sort_values(by=[0])
  labels = Kmean.labels_

  return labels, center

KMeans Clustering finding Centers with SD

In [10]:
def getSD(data):

  sd_set = []
  x = []
  y = []

  col_list = []
  num = 0
  for col in data.columns:
    if num <= batch:
      for row in data.index:
        if data.loc[row,col] != 0: col_list.append(row)
      if len(col_list) == 0:
        continue
      else:
        num += 1
    else:
      mean = sum(col_list)/len(col_list)
      sd_list = [(i-mean)**2 for i in col_list]
      sd = sum(sd_list)/len(sd_list)
      sd_set.append([col, sd] * batch)
      col_list = []
      num = 0
  
  for d in sd_set:
    x.append(d[0])
    y.append(d[1])

  

  Kmean = KMeans(n_clusters=3)
  Kmean.fit(sd_set)
  center = pd.DataFrame(Kmean.cluster_centers_).sort_values(by=[0])
  labels = Kmean.labels_

  fig1 = go.Figure()
  fig1.add_trace(go.Scatter(mode='markers', marker={"color": labels}, x=x, y=y))
  fig1.add_trace(go.Scatter(mode='markers', marker={"size": 20}, x=center.iloc[:,0], y=center.iloc[:,1]))
  fig1.show()

  return labels, center

Cut Beginning and Ending from DataFrame

In [11]:
def removeStartEnd(data, centers, labels):
  start = (np.abs(data.columns - centers.iloc[0,0])).argmin()
  end = (np.abs(data.columns - centers.iloc[2,0])).argmin()
  i1 = 0
  i2 = 0
  for s in range(start, len(labels)):
    if labels[s] != labels[s+1]:
      i1 = s
      break
  r = range(end, start, -1)
  for s in r:
    if labels[s] != labels[s-1]:
      i2 = s
      break
  return data.iloc[:, i1:i2]

Find Separation line

In [12]:
def findRegressionLine(data):

  x_filtered = []
  y_filtered = []
  for col in data.columns:
    for row in data.index:
      if data.loc[row,col] != 0:
        x_filtered.append(col)
        y_filtered.append(row)

  x_fil = np.array(x_filtered)
  y_fil = np.array(y_filtered)

  fit = np.polyfit(x_fil,y_fil, 1)

  return fit, x_fil, y_fil

Binding Filter with top bottom separation

In [13]:
def bindFilter(df_f, row, col, row_len, col_len, sizeH, sizeV):
  val = df_f.iloc[row,col]

  r0 = row - sizeV
  r1 = row + sizeV + 1
  if r0 < 0 : r0 = 0
  if r1 > row_len: r1 = row_len
  c0 = col - sizeH
  c1 = col + 1 + sizeH
  if c0 < 0 : c0 = 0
  if c1 > col_len: c1 = col_len
  
  fil_frame = df_f.iloc[r0:r1, c0:c1]

  frame_med = fil_frame.sum().sum()/fil_frame.size
  if fil_frame.sum().sum() != val:
    fil_frame = fil_frame + frame_med


  return fil_frame, r0, r1, c0, c1

def filterBind(df0, sizeH, sizeV, top, bottom, y_fit):
  df_fil = copy.deepcopy(df0)
  x_len = len(x)
  df_max = df_fil.max().max()
  col_len = len(df_fil.columns)
  row_len = len(df_fil.index)
  col_range = range(col_len)
  row_range = range(row_len)
  for row in row_range:
    for col in col_range:
      if df0.iloc[row,col] == 0:
        pass
      #separating: only top
      elif (df0.index[row] < y_fit[col]) and top:
        df_fil.iloc[row,col] = 0
      elif (df0.index[row] > y_fit[col]) and bottom:
        df_fil.iloc[row,col] = 0
      else:
        frame, r0, r1 ,c0, c1 = bindFilter(df0, row, col, row_len, col_len, sizeH, sizeV)
        df_fil.iloc[r0:r1,c0:c1] = frame 

  return df_fil

Extract Trendlines from Grid

In [14]:
def extractLineFromGrid(data, limit):
  col_len = len(data.columns)
  row_len = len(data.index)
  col_range = range(col_len)
  row_range = range(row_len)
  ind = data.index
  x_l = []
  y_l = []

  dsum = data.sum().sum()
  cels = 0
  for col in col_range:
    for row in row_range:
      if data.iloc[row,col] != 0:
        cels += 1

  aver = dsum/cels

  for col in col_range:
    c = []
    for row in row_range:
      d = data.iloc[row,col]
      if d != 0: 
        c.append([ind[row],d])

    summe = []
    scale = 0
    for p in c:
      z = p[0] * p[1]
      summe.append(z)
      scale += p[1]
    if (len(c) != 0) and (scale > aver*limit):
      res = sum(summe)/scale
      y_l.append(res)
      x_l.append(data.columns[col])
  return x_l, y_l

Line Filter

In [15]:
def addFilter(x_old, y_old, z_old, parameter):
    x = x_old
    y = y_old
    z = z_old

    for par in parameter:
        if par["onOff"][0] == "REMOVE REPEATING POINTS":
            x, y, z = removeRepeating(x, y, z)
    return x, y, z

def addVisualParameter(x_old, y_old, z_old, par):
    x = x_old
    y = y_old
    z = z_old

    if par["type"] == "gaussian":
        x, y, z = gaussian(x, y, z, par["value"])

    if par["type"] == "sideBySide":
        x, y, z = sideBySide(x, y, z, par["value"])

    if par["type"] == "oneOnOther":
        x, y, z = oneOnOther(x, y, z, par["value"])
    return x, y, z



def removeRepeating(x, y, z):
    ##### remove repeating values and take the one in the middle ####
    empty = [False] * len(y)
    empty[0] = True
    empty[-1] = True
    ind = []
    for i in range(len(y) - 1):
        if y[i] != y[i + 1]:
            empty[i + 1] = True
            ind.append(i + 1)
    abs = []
    for n in range(len(ind) - 1):
        abs.append(int(np.floor((ind[n + 1] - ind[n]) / 2)))
    abs.append(0)
    indexes = [a + b for a, b in zip(abs, ind)]

    emptyX = [False] * len(y)
    emptyX[0] = True
    emptyX[-1] = True
    for m in indexes:
        emptyX[m] = True
    x = x[emptyX]
    y = y[emptyX]
    return x.tolist(), y.tolist(), z

def gaussian(x, y, z, sigma):
    y_new = gaussian_filter1d(y, float(sigma), mode="nearest")
    return x, y_new, z

def sideBySide(x, y, z, ind):
    ind = int(ind)
    y_new = []
    for val in range(len(y)):
        i1 = val if (val-ind < 0) else val - ind
        i2 = val if (val+ind >= len(y)) else val + ind
        lst = y[i1:i2]
        v = sum(lst)/len(lst)
        y_new.append(v)
    return x, y_new, z

def oneOnOther(x, y, z, ind):
    ind = int(ind)
    y_new = []
    x_new = []
    i = 0
    while i < len(y)-ind:
      i2 = i + ind
      index = int(i2 - np.floor(ind/2))
      lst = y[i:i2]
      v = sum(lst)/len(lst)
      y_new.append(v)
      x_new.append(x[index])
      i = i2
    if i < len(y):
      lst = y[i:len(y)]
      index = int(len(y) - np.floor(len(lst) / 2)) -1
      print(index)
      print(len(x))
      v = sum(lst) / len(lst)
      y_new.append(v)
      x_new.append(x[index])

    return x_new, y_new, z 

Get Middle Lines from Top Scan

In [16]:
def getMiddleLines(data):
  middle = (np.abs(data.columns.astype("float64")-0)).argmin()
  top_data = pd.Series(data.iloc[:,middle]).dropna()
  top_data_plus = pd.Series(data.iloc[:,middle+1]).dropna()
  top_data_minus = pd.Series(data.iloc[:,middle-1]).dropna()
  return top_data, top_data_plus, top_data_minus

Get selected line from top scan

In [17]:
def getSelectedLine(data, shift):
  middle = (np.abs(data.columns.astype("float64")-0)).argmin()
  top_data = pd.Series(data.iloc[:,middle + shift]).dropna()
  return top_data

Fit Polyline

In [18]:
def fitPolynom(data):
	def objective(x, a, b, c, d, e, f, g, h, i, j, k, l, m):
		return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e * x**5) + (f * x**6) + (g * x**7) + (h * x**8) + (i * x**9) + (j * x**10) + (k * x**11) + (l * x**12) + m

	popt, _ = curve_fit(objective, data.index, data)
	a, b, c, d, e, f, g, h, i, j, k, l, m = popt

	x_line = np.arange(len(data.index))
	y_line = objective(x_line, a, b, c, d, e, f, g, h, i, j, k, l, m)

	return x_line, y_line

Read JSON and create Corrected Values


In [19]:
df_original = readJson("/content/drive/MyDrive/100__2021-03-29094210-932382__0.json", "L2")
start, end = findStartEnd(df_original)
(x, y, st, en) = create1DXCorrected(df_original, 7/60, start, end)

#fig = make_subplots(rows=1, cols=2)
#fig.add_trace(go.Scatter(x=x, y=y, mode='markers'), row=1, col=1)
#fig.add_trace(go.Histogram(y=y), row=1, col=2)
#fig.show()

Reading Json... 


Create and fill grid

In [20]:
grid_df, x_space, y_space = createGrid(x, y, df_original)
filled_grid = fillEmptyGrid(grid_df, x, y, x_space, y_space)

#fig = px.imshow(filled_grid, color_continuous_scale="Greys", range_color=[0,1])
#fig.show()

x, y, z = gridTo3D(filled_grid)
fig = go.Figure(data=go.Heatmap(x=x, y=y, z=z, type='heatmap', colorscale = 'Viridis'))
fig.show()

Filter Noise

In [21]:
fil1_data = filterNoise(filled_grid, 0)

#fig = px.imshow(fil1_data, color_continuous_scale="Greys", range_color=[0,1])
#fig.show()

x, y, z = gridTo3D(fil1_data)
fig = go.Figure()
fig.add_trace(go.Heatmap(x=x, y=y, z=z, type='heatmap', colorscale = 'Viridis'))
fig.add_trace(go.Scatter(x=[min(x),st,st, min(x)], y=[min(y), min(y), max(y), max(y)], fill="toself", fillcolor="LightSalmon", opacity=0.5, line_width=0))
fig.add_trace(go.Scatter(x=[en,max(x),max(x),en], y=[min(y), min(y), max(y), max(y)], fill="toself", fillcolor="LightSalmon", opacity=0.5, line_width=0, marker={"color": "LightSalmon"}))
fig.show()

Remove Start End

In [22]:
fil1_mid_data = fil1_data.loc[:,st:en]

Make Clusters

In [23]:
#labels, centers = clusterAndFindCenter(fil1_data)
#labels, centers = clusterSDAndFindCenter(fil1_data, 5)




Remove Beginning and End using cluster centers

In [24]:
#fil1_mid_data = removeStartEnd(fil1_data, centers, labels)

#fig = px.imshow(fil1_mid_data, color_continuous_scale="Greys", range_color=[0,1])
#fig.show()

#x, y, z = gridTo3D(fil1_mid_data)
#fig = go.Figure(data=go.Heatmap(x=x, y=y, z=z, type='heatmap', colorscale = 'Viridis'))
#fig.show()

Find Polyfit Regression Separation Line

In [25]:
fit, x_fil1, y_fil1 = findRegressionLine(fil1_mid_data)
y_separation_line = fit[0]*x_fil1 + fit[1]

fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers', x=x_fil1, y=y_fil1))
fig.add_trace(go.Scatter(x=x_fil1, y=y_separation_line))
fig.show()

Filter Top/Bottom and enhance

In [26]:
fil2_top_data = filterBind(fil1_data, 1, 1, True, False, y_separation_line) #Top Filter for Extracting top line from grid
fil2_top_mid_data = filterBind(fil1_mid_data, 1, 1, True, False, y_separation_line) #Top Filter
fil2_bottom_data = filterBind(fil1_data, 1, 1, False, False, y_separation_line) #Bottom Filter

for row in fil2_top_mid_data.index:
  for col in fil2_top_mid_data.columns:
    cel = fil2_top_mid_data.loc[row,col]
    if cel != 0:
      fil2_bottom_data.loc[row,col] = 0


x, y, z = gridTo3D(fil2_bottom_data)
fig = go.Figure(data=go.Heatmap(x=x, y=y, z=z, type='heatmap', colorscale = 'Viridis'))
fig.show()
x, y, z = gridTo3D(fil2_top_mid_data)
fig = go.Figure(data=go.Heatmap(x=x, y=y, z=z, type='heatmap', colorscale = 'Viridis'))
fig.show()

Extract Line from Grid

In [27]:
# TOP
x_l, y_l = extractLineFromGrid(fil2_top_data, 0.4)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_l, y=y_l))
fig.show()

# BOTTOM
x_lb, y_lb = extractLineFromGrid(fil2_bottom_data, 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_lb, y=y_lb))
fig.show()

Filter Line

In [28]:
# TOP
x_f, y_f, z_f = addVisualParameter(x_l, y_l, 0, {"type": "oneOnOther", "value": 3})
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_f, y=y_f, line_shape='spline'))
fig.show()
# BOTTOM
x_fb, y_fb, z_fb = addVisualParameter(x_lb, y_lb, 0, {"type": "oneOnOther", "value": 4})
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_fb, y=y_fb, line_shape='spline'))
fig.show()


189
191


228
229


TOP Scan Read and get middle lines


In [29]:
df_top = readJson("/content/drive/MyDrive/100__2021-03-29094210-932382__1.json", "L2")
df_top = df_top.drop("Line Start [us]", axis=1).drop("Line End [us]", axis=1).reset_index(drop=True).replace("", float("nan")).astype("float64")

top_line_1, top_line_2, top_line_3 = getMiddleLines(df_top)

Reading Json... 


Polynom Fit

In [30]:
x_ployline, y_polyline = fitPolynom(top_line_1)

In [31]:
# make x scaled for cross scan
x_grid_values = fil2_top_data.columns
x_max = max(x_grid_values)
x_min = min(x_grid_values)
y_point_count = len(top_line_1)
x_range = list(np.linspace(x_min, x_max, y_point_count))

# TOP
grid_dff, x_space, y_space = createGrid(x, y, df_original)
filled_grid_top = fillEmptyGrid(grid_dff, x_range, list(top_line_1), x_space, y_space)
xg, yg, zg = gridTo3D(filled_grid_top)

x1, y1, z1 = gridTo3D(fil1_data)

fig = go.Figure()
#fig.add_trace(go.Heatmap(x=x, y=y, z=z, type='heatmap', colorscale = 'greens'))
fig.add_trace(go.Heatmap(x=x1, y=y1, z=z1, type='heatmap', colorscale = 'greens'))
fig.add_trace(go.Scatter(x=list(x_range), y=top_line_3, line_shape='spline'))
fig.add_trace(go.Scatter(x=x_f, y=y_f, line_shape='spline'))
fig.add_trace(go.Scatter(x=x_fb, y=y_fb, line_shape='spline'))

fig.show()